ΙΩΑΝΝΗΣ ΓΙΑΝΝΑΚΟΣ 4970

# Βήμα 1

In [34]:
import pandas as pd
import json

# Διάβασμα του αρχείου με τα δεδομένα
with open('meta_Cell_Phones_and_Accessories.json') as file:
    data = file.readlines()

valid_data = [] # Αρχικοποίηση λίστας για τα τελικά δεδομένα και το σύνολο για τα μοναδικά asin
seen_asins = set()

# Ελέγχος κάθε JSON αντικειμένου
for line in data:
    product = json.loads(line)
    
    # Ελέγχος αν η κατηγορία του καθε προιόντος είναι έγκυρη && αν η περιγραφή κάθε αντικειμένου είναι κενή
    if ("Chargers & Power Adapters" in product["category"] or "Car Chargers" in product["category"]) and (len(product["description"])>0):
        
        if "Chargers & Power Adapters" in product["category"][-1] or "Car Chargers" in product["category"][-1]:
            #  έλεγχος αν το asin έχει ήδη προστεθεί, δλδ αν έχω ξαναδεί το συγκεκριμένο προιόν 
            asin = product["asin"]
            if asin in seen_asins:
                continue  # αν ναι τότε το προσπερνάω
            else:
                seen_asins.add(asin)  # αλλιώς το προσθέτω στο σύνολο για μελλοντικό έλεγχο


            # Συνένωση της περιγραφής των προιόντων 
            description = ' '.join(product["description"])

            # String και Category στη λίστα με τα τελικά δεδομένα
            valid_data.append((description, product["category"][-1]))

# Δημιουργία DataFrame
df = pd.DataFrame(valid_data, columns=['Description', 'Category'])

# Εκτύπωση των πρώτων 10 δεδομένων για έλεγχο
print(df.tail(10))


                                            Description  \
6997  <b>MICTUNING SAE to USB Cable Adapter 3.1A Dua...   
6998  <b>Beeiee S5 Wireless Charging Receiver</b><br...   
6999  <b>ONSON Dual USB Car Charger with safety Hamm...   
7000  VINTRONS Car Charger, 10.5W/2.1A, with 1 Unive...   
7001                               only for freway vr01   
7002  <b>Our promise to you</b><br><br> All HAVIT pr...   
7003  <b>Fitian Magnetic DC Charging Cable cord for ...   
7004  <br>[Apple MFi Certified] 3Pack SEGMOI Lightni...   
7005  This travel adapter has 5 built-in plug-ins (i...   
7006  <b>Technical Specification</b><br> Model: Y020...   

                       Category  
6997  Chargers & Power Adapters  
6998  Chargers & Power Adapters  
6999               Car Chargers  
7000               Car Chargers  
7001  Chargers & Power Adapters  
7002               Car Chargers  
7003               Car Chargers  
7004               Car Chargers  
7005  Chargers & Power Adapters  
7006      

# Βήμα 2

In [35]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Διάβασμα δεδομένων απο json αρχείο και χωρισμός των κατηγοριών και περιγραφών σε δύο λίστες 
description = []
categories = []
 
with open('meta_Cell_Phones_and_Accessories.json') as f:
    for line in f:
        product = json.loads(line)
        
        if "Chargers & Power Adapters" in product["category"] or "Car Chargers" in product["category"]:
            
            if "Chargers & Power Adapters" in product["category"][-1] or "Car Chargers" in product["category"][-1]:

                description.append(' '.join(product["description"]))
                categories.append(product["category"][-1])


# Αρχικοποίηση kfold και tf-idf 
vectorizer = TfidfVectorizer()
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Αρχικοπο΄ίηση των τριών classifier
logistic_regression = LogisticRegression(max_iter=1000)
svm = SVC()
mlp = MLPClassifier(max_iter=1000)


# Συνάρτηση που εκπαιδεύει εναν classifier και επιστρέφει τις επιθυμητές μετρικές
def evaluate_classifier(classifier, x_train, y_train, x_test, y_test):
    classifier.fit(x_train, y_train) # εκπαίδευση του εκάστοτε μοντέλου στο dataset των κατηγοριών και περιγραφών 
    y_prediction = classifier.predict(x_test) # η κατηγορία που προέβλεψε το μοντέλο για την περιγραφή που έλαβε ως input
    
    conf_matrix = confusion_matrix(y_test, y_prediction) # υπολογισμός του confusion matrix 
    accuracy = accuracy_score(y_test, y_prediction) # υπολογισμός της accuracy (μέση τιμή)
    precision = precision_score(y_test, y_prediction, average='macro', zero_division=1) # υπολογισμός του precision (μέση τιμή)
    recall = recall_score(y_test, y_prediction, average='macro', zero_division=1) # υπολογισμός του recall (μέση τιμή)
    F1_measure = f1_score(y_test, y_prediction, average='macro', zero_division=1) # υπολογισμός του f1 (μέση τιμή)
   
    return conf_matrix, accuracy, precision, recall, F1_measure

kf_splits = list(kf.split(description)) # Αποθήκευση των splits του KFold

for train_index, test_index in kf_splits:
    
    # Αποθήκευση των train και test υποσυνόλων του πεδίου description που φτιάχτηκαν με τη μεθοδο KFold
    x_train = np.array(description)[train_index]
    x_test = np.array(description)[test_index]
    
    # Αποθήκευση των train και test υποσυνόλων του πεδίου categories που φτιάχτηκαν με τη μεθοδο KFold
    y_train = np.array(categories)[train_index]
    y_test = np.array(categories)[test_index]
    
    # Εξαγωγή features με χρ΄ήση της tf-idf αναπαράστασης των περιγραφών 
    x_train_tfidf = vectorizer.fit_transform(x_train) 
    x_test_tfidf = vectorizer.transform(x_test)
    
    # Logistic Regression classifier 
    conf_matrix_logistic_regression, accuracy_logistic_regression, precision_logistic_regression, recall_logistic_regression, F1_measure_logistic_regression = evaluate_classifier(logistic_regression, x_train_tfidf, y_train, x_test_tfidf, y_test)

    # SVM classifier 
    conf_matrix_svm, accuracy_svm, precision_svm, recall_svm, F1_measure_svm = evaluate_classifier(svm, x_train_tfidf, y_train, x_test_tfidf, y_test)
    
    # Multi-Layer Perceptron classifier 
    conf_matrix_mlp, accuracy_mlp, precision_mlp, recall_mlp, F1_measure_mlp = evaluate_classifier(mlp, x_train_tfidf, y_train, x_test_tfidf, y_test)
    
    # Εκτύπωση confusion matrix και μετρικ΄ών για Logistic Regression classifier
    print("Logistic Regression:")
    print(conf_matrix_logistic_regression)
    print("Accuracy:", accuracy_logistic_regression)
    print("Precision:", precision_logistic_regression)
    print("Recall:", recall_logistic_regression)
    print("F1-Measure:", F1_measure_logistic_regression)
    print("")

    # Εκτύπωση confusion matrix και μετρικ΄ών για SVM classifier
    print("SVM:")
    print(conf_matrix_svm)
    print("Accuracy:", accuracy_svm)
    print("Precision:", precision_svm)
    print("Recall:", recall_svm)
    print("F1-Measure:", F1_measure_svm)
    print("")
    
    # Εκτύπωση confusion matrix και μετρικ΄ών για Multi-Layer Perceptron classifier
    print("Multi-Layer Perceptron:")
    print(conf_matrix_mlp)
    print("Accuracy:", accuracy_mlp)
    print("Precision:", precision_mlp)
    print("Recall:", recall_mlp)
    print("F1-Measure:", F1_measure_mlp)
    print("")
    
    break 

Logistic Regression:
[[1085   88]
 [ 354  439]]
Accuracy: 0.775178026449644
Precision: 0.7935064541183328
Recall: 0.7392863170817974
F1-Measure: 0.7479662629356352

SVM:
[[1082   91]
 [ 324  469]]
Accuracy: 0.788911495422177
Precision: 0.8035295163584637
Recall: 0.7569230554220702
F1-Measure: 0.7661795610518409

Multi-Layer Perceptron:
[[1057  116]
 [ 360  433]]
Accuracy: 0.757884028484232
Precision: 0.7673244353948219
Recall: 0.7235680060718843
F1-Measure: 0.7307608651870947



Σχολιασμός: Από την παραπάνω εκτύπωση των αποτελεσμάτων, παρατηρούμε ότι υψηλότερο Accuracy (78.90%), Precision (80.35%), καθώς και Recall (75.70%), αλλά και F1 (0.7661) από τους τρεις classifiers έχει ο SVM. Συνεπώς, καταλήγουμε ότι ο SVM classifier είναι ο περισσότερο αποδοτικός για το παρόν πρόβλημα.

In [37]:
# Εκπαίδευση του Logistic Regression στο τελευταίο fold

# Αποθήκευση των train και test υποσυνόλων του πεδίου description που φτιάχτηκαν με τη μεθοδο KFold
x_train = np.array(description)[train_index]
x_test = np.array(description)[test_index]

# Αποθήκευση των train και test υποσυνόλων του πεδίου categories που φτιάχτηκαν με τη μεθοδο KFold
y_train = np.array(categories)[train_index] 
y_test = np.array(categories)[test_index]

# Εξαγωγή features με χρ΄ήση της tf-idf αναπαράστασης των περιγραφών 
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

logistic_regression.fit(x_train_tfidf, y_train) # εκπαίδευση του μοντέλου

# Εύρεση των 20 λέξεων με το μεγαλύτερο θετικό βάρος
feature_names = vectorizer.get_feature_names_out()
top_positive_words = [feature_names[i] for i in logistic_regression.coef_[0].argsort()[-20:][::-1]] # παίρνω τις 20 λέξεις με μεγαλύτερο βάρος και τις ταξινομώ σε φθίνουσα σειρά (ο πίνακας coef περιέχει τα βάρη)

# Εύρεση των 20 λέξεων με το μικρότερο αρνητικό βάρος
top_negative_words = [feature_names[i] for i in logistic_regression.coef_[0].argsort()[:20]] # παίρνω τις 20 λέξεις με χαμηλότερο βάρος και τις ταξινομώ τα σε αύξουσα σειρά (ο πίνακας coef περιέχει τα βάρη)

print("Logistic Regression 20 λέξεις με το μεγαλύτερο θετικό βάρος: ")
print(top_positive_words)
print("")
print("Logistic Regression 20 λέξεις με το μικρότερο αρνητικό βάρος: ")
print(top_negative_words)

Logistic Regression 20 λέξεις με το μεγαλύτερο θετικό βάρος: 
['wall', 'travel', 'ac', 'battery', 'home', '240v', 'desktop', 'dock', '60hz', 'qi', 'receiver', 'original', 'simply', '100', 'lg', 'pack', 'solar', 'of', 'sync', 'is']

Logistic Regression 20 λέξεις με το μικρότερο αρνητικό βάρος: 
['car', 'vehicle', 'lighter', '12v', 'cigarette', '12', 'dc', '24v', 'dual', 'auto', 'indicated', '1a', 'retractable', 'powered', 'led', 'in', 'very', 'road', 'button', 'while']


Οι λέξεις που έχουν το υψηλότερο θετικό βάρος στον Logistic Regression classifier αναφέρονται σε φορτιστές για πρίζες τοίχου (wall), σε φορητούς φορτιστές για ταξίδια (travel), σε φορτιστές που λειτουργούν με εναλλασσόμενο ρεύμα (ac), σε προιόντα που χρησιμοποιούν μπαταρίες (battery), σε πιθανώς προιόντα που προορίζονται για χρήση στο σπίτι (home), σε qi ασύρματη φόρτιση (qi), σε πιθανώς γνήσια προιόντα (original), σε σύνδεση προιόντων με φωτοβολταικά πάνελ (solar), κλπ.
Στο αντίστοιχο αρνητικό βάρος, βλέπουμε ότι οι λέξεις αυτές είναι πιο γενικές και συνήθως δεν προσδιορίζουν τόσο το προϊόν (car, vehicle, lighter, 12v, cigarette, 12, dc, 24v, dual, auto, indicated, 1a, retractable, powered, led, in, very, road, button, while). Αυτές οι λέξεις είναι πιο γενικές και συναντώνται σε μεγαλύτερη ποικιλία προϊόντων και όχι τόσο πολύ στην κατηγορία που μας ενδιαφέρει.

# Βήμα 3

In [40]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec



# Διάβασμα δεδομένων απο json αρχείο και χωρισμός των κατηγοριών και περιγραφών σε δύο λίστες 
description = []
categories = []
 
with open('meta_Cell_Phones_and_Accessories.json') as f:
    for line in f:
        product = json.loads(line)
        
        if "Chargers & Power Adapters" in product["category"] or "Car Chargers" in product["category"]:
            
            if "Chargers & Power Adapters" in product["category"][-1] or "Car Chargers" in product["category"][-1]:

                description.append(' '.join(product["description"]))
                categories.append(product["category"][-1])
                
# Αρχικοποίηση kfold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Αρχικοποίηση των τριών classifier
logistic_regression = LogisticRegression(max_iter=1000)
svm = SVC()
mlp = MLPClassifier(max_iter=1000)


# Συνάρτηση που εκπαιδεύει εναν classifier και επιστρέφει τις επιθυμητές μετρικές
def evaluate_classifier(classifier, x_train, y_train, x_test, y_test):
    classifier.fit(x_train, y_train) # εκπαίδευση του εκάστοτε μοντέλου στο dataset των κατηγοριών και περιγραφών 
    y_prediction = classifier.predict(x_test) # η κατηγορία που προέβλεψε το μοντέλο για την περιγραφή που έλαβε ως input
    
    conf_matrix = confusion_matrix(y_test, y_prediction) # υπολογισμός του confusion matrix 
    accuracy = accuracy_score(y_test, y_prediction) # υπολογισμός της accuracy (μέση τιμή)
    precision = precision_score(y_test, y_prediction, average='macro', zero_division=1) # υπολογισμός του precision (μέση τιμή)
    recall = recall_score(y_test, y_prediction, average='macro', zero_division=1) # υπολογισμός του recall (μέση τιμή)
    F1_measure = f1_score(y_test, y_prediction, average='macro', zero_division=1) # υπολογισμός του f1 (μέση τιμή)
   
    return conf_matrix, accuracy, precision, recall, F1_measure

kf_splits = list(kf.split(description)) # Αποθήκευση των splits του KFold

# Εκπαίδευση του Doc2Vec μοντέλου
documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(description)] # δημιουργία αντικειμένου TaggedDocument για κάθε αντικέιμενο στο description, τα οποία χωρίζονται με το κενό 
d2v_model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, workers=4) # εκπαίδευση μοντέλου με δεδομένα από τη λίστα documents 

for train_index, test_index in kf_splits:
    
    # Αποθήκευση των train και test υποσυνόλων του πεδίου description που φτιάχτηκαν με τη μεθοδο KFold
    x_train = np.array(description)[train_index] 
    x_test = np.array(description)[test_index]
    
    # Αποθήκευση των train και test υποσυνόλων του πεδίου categories που φτιάχτηκαν με τη μεθοδο KFold
    y_train = np.array(categories)[train_index]
    y_test = np.array(categories)[test_index]
    
    # Εξαγωγή των features μέσω embeddings
    x_train_emb = [d2v_model.infer_vector(doc.split()) for doc in x_train]
    x_test_emb = [d2v_model.infer_vector(doc.split()) for doc in x_test]
    
    # Logistic Regression classifier 
    conf_matrix_logistic_regression, accuracy_logistic_regression, precision_logistic_regression, recall_logistic_regression, F1_measure_logistic_regression = evaluate_classifier(logistic_regression, x_train_tfidf, y_train, x_test_tfidf, y_test)

    # SVM classifier 
    conf_matrix_svm, accuracy_svm, precision_svm, recall_svm, F1_measure_svm = evaluate_classifier(svm, x_train_tfidf, y_train, x_test_tfidf, y_test)
    
    # Multi-Layer Perceptron classifier 
    conf_matrix_mlp, accuracy_mlp, precision_mlp, recall_mlp, F1_measure_mlp = evaluate_classifier(mlp, x_train_tfidf, y_train, x_test_tfidf, y_test)
    
    # Εκτύπωση confusion matrix και μετρικ΄ών για Logistic Regression classifier
    print("Logistic Regression:")
    print(conf_matrix_logistic_regression)
    print("Accuracy:", accuracy_logistic_regression)
    print("Precision:", precision_logistic_regression)
    print("Recall:", recall_logistic_regression)
    print("F1-Measure:", F1_measure_logistic_regression)
    print("")

    # Εκτύπωση confusion matrix και μετρικ΄ών για SVM classifier
    print("SVM:")
    print(conf_matrix_svm)
    print("Accuracy:", accuracy_svm)
    print("Precision:", precision_svm)
    print("Recall:", recall_svm)
    print("F1-Measure:", F1_measure_svm)
    print("")
    
    # Εκτύπωση confusion matrix και μετρικ΄ών για Multi-Layer Perceptron classifier
    print("Multi-Layer Perceptron:")
    print(conf_matrix_mlp)
    print("Accuracy:", accuracy_mlp)
    print("Precision:", precision_mlp)
    print("Recall:", recall_mlp)
    print("F1-Measure:", F1_measure_mlp)
    print("")
    
    break 

Logistic Regression:
[[1085   88]
 [ 354  439]]
Accuracy: 0.775178026449644
Precision: 0.7935064541183328
Recall: 0.7392863170817974
F1-Measure: 0.7479662629356352

SVM:
[[1082   91]
 [ 324  469]]
Accuracy: 0.788911495422177
Precision: 0.8035295163584637
Recall: 0.7569230554220702
F1-Measure: 0.7661795610518409

Multi-Layer Perceptron:
[[1046  127]
 [ 351  442]]
Accuracy: 0.7568667344862665
Precision: 0.7627743608259225
Recall: 0.7245538272329601
F1-Measure: 0.7315266516966923



Και με τις δύο προσεγγίσεις τόσο με την TF-IDF, αλλά και με την Doc2Vec οι μετρικές της Logistic Regression και SVM παρέμειναν ίδιες. Ωστόσο, οι μετρικές του Multi-Layer Perceptron ήταν ελαφρώς χειρότερες με τη χρήση του Doc2Vec. Τέλος, τα αποτελέσματα στην Doc2Vec προσέγγιση μπορεί να είναι επηρεασμένα από την επιλογή των παραμέτρων (όπως το μέγεθος του vector).

# Βήμα 4  

δεν επιλύθηκε, παρακάτω είναι κάποιες προσπάθειες

In [50]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)


[==================================================] 100.0% 1662.8/1662.8MB downloaded
C:\Users\ggian/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


In [52]:
import gensim
#path = 'C:\Users\ggian/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz'
g_model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)  

train_gmodel = [[g_model[x] for x in y if x in g_model] for y in x_train_gsim]
train_data_labels = [(x,y) for (x,y) in zip(train_gmodel, y_train) if len(x) > 0]
x_train_gm = [np.array(x) for (x,y) in train_data_labels]
y_train_gm = [y for (x,y) in train_data_labels]

X_train_gmodel = [x.mean(axis = 0) for x in x_train_gm]

test_gmodel = [[g_model[x] for x in y if x in g_model] for y in x_test_gsim]
test_data_labels = [(x,y) for (x,y) in zip(test_gmodel, y_test) if len(x) > 0]
x_test_gm = [np.array(x) for (x,y) in test_data_labels]
y_test_gm = [y for (x,y) in test_data_labels]

x_test_gmodel = [x.mean(axis = 0) for x in x_test_gm]

lr_clf.fit(X_news_train_gmodel, np.array(y_news_train_gm))
lr_clf.score(np.array(X_news_test_gmodel),y_news_test_gm)


300
[-0.05419922  0.01708984 -0.00527954  0.33203125 -0.25       -0.01397705
 -0.15039062 -0.265625    0.01647949  0.3828125  -0.03295898 -0.09716797
 -0.16308594 -0.04443359  0.00946045  0.18457031  0.03637695  0.16601562
  0.36328125 -0.25585938  0.375       0.171875    0.21386719 -0.19921875
  0.13085938 -0.07275391 -0.02819824  0.11621094  0.15332031  0.09082031
  0.06787109 -0.0300293  -0.16894531 -0.20800781 -0.03710938 -0.22753906
  0.26367188  0.012146    0.18359375  0.31054688 -0.10791016 -0.19140625
  0.21582031  0.13183594 -0.03515625  0.18554688 -0.30859375  0.04785156
 -0.10986328  0.14355469 -0.43554688 -0.0378418   0.10839844  0.140625
 -0.10595703  0.26171875 -0.17089844  0.39453125  0.12597656 -0.27734375
 -0.28125     0.14746094 -0.20996094  0.02355957  0.18457031  0.00445557
 -0.27929688 -0.03637695 -0.29296875  0.19628906  0.20703125  0.2890625
 -0.20507812  0.06787109 -0.43164062 -0.10986328 -0.2578125  -0.02331543
  0.11328125  0.23144531 -0.04418945  0.10839844 -

In [53]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
import gensim.downloader as api

# Φόρτωση των GloVe embeddings
glove_model = api.load("glove-wiki-gigaword-300")

# Εξαγωγή των embeddings για κάθε λέξη στο κάθε κείμενο
x_train_emb = [[glove_model[word] for word in doc.split() if word in glove_model] for doc in x_train]
x_test_emb = [[glove_model[word] for word in doc.split() if word in glove_model] for doc in x_test]

# Υπολογισμός του μέσου όρου των embeddings για κάθε κείμενο
x_train_emb_avg = [np.mean(embeddings, axis=0) for embeddings in x_train_emb]
x_test_emb_avg = [np.mean(embeddings, axis=0) for embeddings in x_test_emb]


# Αρχικοποίηση του Logistic Regression classifier
logistic_regression = LogisticRegression(max_iter=1000)

# Εκπαίδευση του Logistic Regression classifier
logistic_regression.fit(x_train_emb_avg, y_train)

# Πρόβλεψη των κατηγοριών για τα test δεδομένα
y_prediction = logistic_regression.predict(x_test_emb_avg)

# Υπολογισμός των μετρικών
conf_matrix_logistic_regression = confusion_matrix(y_test, y_prediction)
accuracy_logistic_regression = accuracy_score(y_test, y_prediction)
precision_logistic_regression = precision_score(y_test, y_prediction, average='macro', zero_division=1)
recall_logistic_regression = recall_score(y_test, y_prediction, average='macro', zero_division=1)
F1_measure_logistic_regression = f1_score(y_test, y_prediction, average='macro', zero_division=1)

# Εκτύπωση των μετρικών
print("Logistic Regression:")
print(conf_matrix_logistic_regression)
print("Accuracy:", accuracy_logistic_regression)
print("Precision:", precision_logistic_regression)
print("Recall:", recall_logistic_regression)
print("F1-Measure:", F1_measure_logistic_regression)


[==================================================] 100.0% 376.1/376.1MB downloaded


C:\Users\ggian\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\ggian\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (7860,) + inhomogeneous part.